In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from os.path import join
import datetime

from models.base_line import BasicModel
from training.trainer_base import Trainer
from data.transforms import ResizeTransform
from modules.cross_validation import MyKFoldCV
from modules.dataset_handling import DatasetHandler
from modules.logging import MyLogger
from utils.metrics import compute_confusion_matrices, compute_accuracy, compute_f1score
from utils.visulisation import visualize_confusion_matrices


torch.manual_seed(42)


in_size = 128
out_size = 2

project_path = "/home/gatemrou/uds/Eye_Tracking/"
assert(os.path.exists(project_path))

In [ ]:
data_type = 'scanpath'       # scanpath or heatmap
scanpath_type = 'b'       # a, b or c
model_type = 'base'      # def_cnn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if model_type == 'base':
    model = BasicModel(in_size, out_size)
print('using device: ' + device.__str__())
# Create a dataset from image folder
root = join(project_path, 'data/datasets/{}s/'.format(data_type))
if data_type == 'scanpath':
    root = join(root, '{}/'.format(scanpath_type))

transform = ResizeTransform(in_size, in_size)

dataset = DatasetHandler(root, transform)

outer_kfolds = 8
inner_kfolds = 7
batch_size = 32

outer_cv = MyKFoldCV(dataset, outer_kfolds, create_train_loaders=False, batch_size=batch_size)

learning_rate = 0.02

# Define the loss function
loss = nn.CrossEntropyLoss()

logger = MyLogger(join(project_path, 'experiments/{}s{}_{}_{}/'.format(data_type, scanpath_type, model_type, in_size)))

cur_time = str(datetime.datetime.now())
# Define trainer 
if data_type == 'scanpath':
    save_path = join(project_path, 'saved_models/{}s_{}_{}_{}/'.format(data_type, scanpath_type, model_type, cur_time))
else:
    save_path = join(project_path, 'saved_models/{}s_{}_{}/'.format(data_type, model_type, cur_time))

trainer = Trainer(model, save_path, loss, logger=logger)

In [ ]:
num_epochs = 5

for i in range(outer_cv.k):
    print(f'outer fold: {i+1}')
    trainer.reset()
    logger.log_new_fold(i, outer_cv.get_test_participants())
    if model_type == 'base':
        model = BasicModel(in_size, out_size)
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    trainer.set_optimizer(optimizer)
    trainer.set_model(model)
    trainer.set_save_path(join(save_path, f'fold_{i+1}'))
    inner_cv = MyKFoldCV(dataset, inner_kfolds, ignore=outer_cv.get_test_participants(), batch_size=batch_size)
    print('testing on: ', outer_cv.get_test_participants())
    print('training on: ', inner_cv.get_participants())
    # training
    for j in range(inner_cv.k):
        print(f'inner fold: {j+1}')
        train_loader = inner_cv.get_train_loader()
        test_loader = inner_cv.get_test_loader()
        print('validating on: ', inner_cv.get_test_participants())
        trainer.set_train_loader(train_loader)
        trainer.set_val_loader(test_loader)
        for _ in range(num_epochs):
            trainer.run_epoch()
        inner_cv.next_split()
    trainer.save_best_model()
    trainer.load_best_model()
    # testing
    test_loader = outer_cv.get_test_loader()
    test_model = trainer.model
    test_model.eval()
    for p in outer_cv.get_test_participants():
        assert(p not in inner_cv.get_participants())
    acc = compute_accuracy(test_model, test_loader)
    f1 = compute_f1score(test_model, test_loader)
    cfs = compute_confusion_matrices(test_model, dataset, outer_cv.get_test_participants())
    logger.log_test_metrics(acc, f1, cfs)
    outer_cv.next_split()
logger.log_f1_history()



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

for imgs, labs in outer_cv.get_test_loader():
    i = 1
    lab = labs[i]
    input_image = imgs[i].clone().detach().requires_grad_(True)
    # Set the model to evaluation mode
    trainer.model.eval()
    trainer.model.cpu()
    
    # Forward pass to get the output
    output = trainer.model.forward(input_image.unsqueeze(0))  # Adding batch dimension
    _, pred = torch.max(output, 1)
    # Calculate gradients of the output with respect to the input image
    output.squeeze()[pred].backward()
    print(output)
    # Get the gradients from the input image
    gradients = input_image.grad.squeeze().numpy()
    gradients[gradients<0] = 0
    gradients = gradients / np.max(gradients)

    # Plot the gradients
    plt.figure(figsize=(8, 6))
    plt.imshow(np.transpose(gradients, (1, 2, 0)) )
    plt.title(f'Gradients: {pred.item()}')
    plt.axis('off')
    plt.show()
    plt.figure(figsize=(8, 6))
    plt.imshow(np.transpose(input_image.detach().cpu().numpy(), (1, 2, 0)) )
    plt.title(f'Input image: {lab}')
    plt.axis('off')
    plt.show()
    
    

In [ ]:
trainer.save_best_model()
print('testing on: ', outer_cv.get_test_participants())
trainer.test(outer_cv.get_test_loader())

In [ ]:
test_model = BasicModel(in_size, out_size)
test_model.load_state_dict(torch.load(join(project_path, 'saved_models/scanpaths_base/model_epoch_16.pkl')))
test_pp = ['10', '11', '13', '14']

In [ ]:
print(compute_f1score(test_model, dataset.get_loader(test_pp)))

In [ ]:
print(compute_f1score(test_model, dataset.get_loader(test_pp)))

In [ ]:
cfs = compute_confusion_matrices(test_model, dataset, test_pp)

In [ ]:
visualize_confusion_matrices(cfs)

In [ ]:
trainer.logger = None
trainer.reset()
if model_type == 'base':
    model = BasicModel(in_size, out_size)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
trainer.set_optimizer(optimizer)
trainer.set_model(model)
inner_cv = MyKFoldCV(dataset, inner_kfolds, ignore=outer_cv.get_test_participants(), batch_size=batch_size)
print('testing on: ', outer_cv.get_test_participants())
print('training on: ', inner_cv.get_participants())
train_loader = inner_cv.get_train_loader()
test_loader = inner_cv.get_test_loader()
print('validating on: ', inner_cv.get_test_participants())
trainer.set_train_loader(train_loader)
trainer.set_val_loader(test_loader)

In [ ]:
train_loader = inner_cv.get_train_loader()
test_loader = inner_cv.get_test_loader()
trainer.set_train_loader(train_loader)
trainer.set_val_loader(test_loader)
for _ in range(20):
    trainer.run_epoch()

In [ ]:
trainer.save_best_model()